<a href="https://colab.research.google.com/github/lecielpourlamer/01-thymeleaf-demo-employees-list-thymeleafdemo/blob/master/handson_llm_ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Phi-3 모델과의 호환성 때문에 transformers 4.48.3 버전을 사용
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [2]:
# 깃허브에서 위제 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정
from transformers.utils import logging

logging.disable_progress_bar()

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 모델과 토크나이저를 로드
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added mal

In [4]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


In [5]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

# 확률 분포로부터 하나의 토큰 선택하기(샘플링/디코딩)

In [6]:
prompt = "The capital of France is"

# 입력 프롬프트를 토근화하기
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# 입력 토큰을 GPU에 배치하기
input_ids = input_ids.to("cuda")

# lm_head 앞에 있는 model의 출력을 얻기
model_output = model.model(input_ids)

# lm_head의 출력을 얻기
lm_head_output = model.lm_head(model_output[0])

In [7]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

In [8]:
# 모델의 출력 행렬의 차원 확인
model_output[0].shape

torch.Size([1, 5, 3072])

In [9]:
lm_head_output.shape

torch.Size([1, 5, 32064])

# 키와 값을 캐싱하여 생성 속도 높이기

In [10]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# 입력 프롬프트를 토큰화하기
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [11]:
# 캐싱을 사용하여 100개의 토큰을 생성하는 데 걸리는 시간 측정하기
# %%timeit 매직 명령은 기본적으로 코드를 1,000번 반복하는 과정을 7번 수행한 후 평균을 낸다.
# 전자의 횟수는 -n 옵션으로, 후자의 횟수는 -r 옵션으로 지정

%%timeit -n 1

# 텍스트를 생성
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True,
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


7.33 s ± 259 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# 캐싱을 사용하지 않을 경우 걸리는 시간 측정하기

%%timeit -n 1

# 텍스트를 생성
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False,
)

33.4 s ± 343 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
